# Capability Tokens — Scoped, Time-Limited Secret Access

SafeAI issues **capability tokens** that grant agents narrow, time-bound permission to
invoke tools and resolve secrets. Each token is bound to:

| Field | Purpose |
|---|---|
| **agent_id** | The agent that owns the token |
| **tool_name** | The single tool the token authorises |
| **actions** | Allowed actions (e.g. `invoke`) |
| **secret_keys** | Which secret keys the token may resolve |
| **ttl** | Time-to-live — the token expires after this duration |
| **session_id** | Optional session binding |

This notebook walks through issuing, validating, revoking, and resolving secrets
with capability tokens.

In [1]:
import tempfile
from pathlib import Path
from click.testing import CliRunner
from safeai.cli.init import init_command
from safeai import SafeAI

tmp = tempfile.TemporaryDirectory()
work = Path(tmp.name)
CliRunner().invoke(init_command, ["--path", str(work)])
ai = SafeAI.from_config(work / "safeai.yaml")
print(f"SafeAI loaded from {work}")

SafeAI loaded from /var/folders/w6/vcgrptb532z_npwq5m8kmn880000gp/T/tmpjroxt4vq


## 1. Issue a capability token

Issue a token for agent `bot-1` to invoke the `api_call` tool, with access to the
`API_KEY` secret. The token is valid for 10 minutes by default.

In [2]:
token = ai.issue_capability_token(
    agent_id="bot-1",
    tool_name="api_call",
    actions=["invoke"],
    ttl="10m",
    secret_keys=["API_KEY"],
)

print(f"token_id    = {token.token_id}")
print(f"agent_id    = {token.agent_id}")
print(f"tool_name   = {token.scope.tool_name}")
print(f"actions     = {token.scope.actions}")
print(f"secret_keys = {token.scope.secret_keys}")
print(f"issued_at   = {token.issued_at}")
print(f"expires_at  = {token.expires_at}")

token_id    = cap_4987d63b099f4543b0175058
agent_id    = bot-1
tool_name   = api_call
actions     = ['invoke']
secret_keys = ['API_KEY']
issued_at   = 2026-02-21 11:15:17.273444+00:00
expires_at  = 2026-02-21 11:25:17.273444+00:00


## 2. Validate the token

Validation checks that the token exists, is not expired or revoked, and that the
requesting agent, tool, and action all match the token's scope.

In [3]:
# Correct agent and tool — should be valid
result = ai.validate_capability_token(
    token.token_id,
    agent_id="bot-1",
    tool_name="api_call",
    action="invoke",
)
print(f"Correct agent + tool:  allowed={result.allowed}, reason={result.reason!r}")
print()

# Wrong agent — should be denied
result_bad_agent = ai.validate_capability_token(
    token.token_id,
    agent_id="bot-999",
    tool_name="api_call",
    action="invoke",
)
print(f"Wrong agent:           allowed={result_bad_agent.allowed}, reason={result_bad_agent.reason!r}")
print()

# Wrong tool — should be denied
result_bad_tool = ai.validate_capability_token(
    token.token_id,
    agent_id="bot-1",
    tool_name="delete_db",
    action="invoke",
)
print(f"Wrong tool:            allowed={result_bad_tool.allowed}, reason={result_bad_tool.reason!r}")

Correct agent + tool:  allowed=True, reason='capability token valid'

Wrong agent:           allowed=False, reason='capability token agent binding mismatch'

Wrong tool:            allowed=False, reason='capability token tool binding mismatch'


## 3. Revoke a token

Tokens can be revoked immediately. Once revoked, all subsequent validations fail.

In [4]:
revoked = ai.revoke_capability_token(token.token_id)
print(f"Revoked: {revoked}")
print()

# Validate after revocation — now invalid
result_after = ai.validate_capability_token(
    token.token_id,
    agent_id="bot-1",
    tool_name="api_call",
    action="invoke",
)
print(f"After revocation:  allowed={result_after.allowed}, reason={result_after.reason!r}")

Revoked: True

After revocation:  allowed=False, reason="capability token 'cap_4987d63b099f4543b0175058' is revoked"


## 4. Resolve secrets with a token

Capability tokens gate access to secrets. The `resolve_secret` method validates the
token, checks that the requested key is in the token's `secret_keys` scope, and
fetches the value from a registered backend.

Here we use the built-in `EnvSecretBackend` which reads from environment variables.

In [5]:
import os

# Set an environment variable to serve as our secret
os.environ["TEST_SECRET"] = "my-secret-value"

# Issue a fresh token with access to TEST_SECRET
secret_token = ai.issue_capability_token(
    agent_id="bot-1",
    tool_name="api_call",
    actions=["invoke"],
    ttl="5m",
    secret_keys=["TEST_SECRET"],
)
print(f"Issued token: {secret_token.token_id}")
print(f"Secret keys:  {secret_token.scope.secret_keys}")
print()

# Resolve the secret — the env backend is registered by default
resolved = ai.resolve_secret(
    token_id=secret_token.token_id,
    secret_key="TEST_SECRET",
    agent_id="bot-1",
    tool_name="api_call",
    action="invoke",
    backend="env",
)
print(f"Resolved value:   {resolved.value!r}")
print(f"Backend used:     {resolved.backend}")
print(f"Bound to token:   {resolved.token_id}")
print(f"Bound to agent:   {resolved.agent_id}")

# Clean up the environment variable
del os.environ["TEST_SECRET"]

Issued token: cap_f18b47eb86a340c394b2c587
Secret keys:  ['TEST_SECRET']

Resolved value:   'my-secret-value'
Backend used:     env
Bound to token:   cap_f18b47eb86a340c394b2c587
Bound to agent:   bot-1


## 5. Scope enforcement — wrong secret key denied

If a token only grants access to `API_KEY`, attempting to resolve `OTHER_KEY` is
rejected. The token's `secret_keys` scope acts as an allowlist.

In [6]:
from safeai.secrets.base import SecretAccessDeniedError

# Issue a token that only allows API_KEY
scoped_token = ai.issue_capability_token(
    agent_id="bot-1",
    tool_name="api_call",
    actions=["invoke"],
    ttl="5m",
    secret_keys=["API_KEY"],
)
print(f"Token allows: {scoped_token.scope.secret_keys}")
print()

# Try to resolve a key NOT in scope
try:
    ai.resolve_secret(
        token_id=scoped_token.token_id,
        secret_key="OTHER_KEY",
        agent_id="bot-1",
        tool_name="api_call",
        action="invoke",
        backend="env",
    )
except SecretAccessDeniedError as exc:
    print(f"Denied: {exc}")
    print()
    print("The token's secret_keys scope prevented access to an unauthorised key.")

Token allows: ['API_KEY']

Denied: capability token does not allow secret key 'OTHER_KEY'

The token's secret_keys scope prevented access to an unauthorised key.


## 6. TTL and expiry

Every token carries an `expires_at` timestamp derived from the TTL (e.g. `10m`, `1h`, `30s`).
Once expired, the token fails validation automatically.

Use `purge_expired_capability_tokens()` to clean up expired and revoked tokens from
the in-memory store.

In [7]:
# Issue a short-lived token to illustrate TTL
short_token = ai.issue_capability_token(
    agent_id="bot-1",
    tool_name="api_call",
    actions=["invoke"],
    ttl="30s",
    secret_keys=["API_KEY"],
)
print(f"Token issued at:  {short_token.issued_at}")
print(f"Token expires at: {short_token.expires_at}")
print(f"TTL window:       {short_token.expires_at - short_token.issued_at}")
print()

# Purge expired and revoked tokens
purged = ai.purge_expired_capability_tokens()
print(f"Purged tokens: {purged}")
print()
print("Tokens that have passed their expires_at or been revoked are removed.")
print("In production, call purge_expired_capability_tokens() periodically to")
print("keep the in-memory token store lean.")

Token issued at:  2026-02-21 11:15:17.291243+00:00
Token expires at: 2026-02-21 11:15:47.291243+00:00
TTL window:       0:00:30

Purged tokens: 1

Tokens that have passed their expires_at or been revoked are removed.
In production, call purge_expired_capability_tokens() periodically to
keep the in-memory token store lean.


In [8]:
# Cleanup
tmp.cleanup()
print("Temporary directory cleaned up.")

Temporary directory cleaned up.


---

## Summary

| Concept | Takeaway |
|---|---|
| **Issue** | `issue_capability_token()` creates a scoped, time-limited token |
| **Validate** | `validate_capability_token()` checks agent, tool, action, session, and expiry |
| **Revoke** | `revoke_capability_token()` immediately invalidates a token |
| **Resolve secrets** | `resolve_secret()` validates the token and fetches from a registered backend |
| **Scope enforcement** | Only `secret_keys` listed in the token are accessible |
| **TTL** | Tokens auto-expire; `purge_expired_capability_tokens()` cleans up the store |

Capability tokens ensure that agents operate under the **principle of least privilege** —
each token grants only the minimum access needed for a specific tool invocation, and
that access is automatically revoked when the TTL expires.